In [1]:
from __future__ import division, print_function, absolute_import

# 载入MNIST数据集
from tensorflow.examples.tutorials.mnist import input_data
#读取数据集，
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf

# 设置训练参数
learning_rate = 0.001 #学习率
num_steps = 200 #训练步数
batch_size = 128 #每次训练的数据集大小

# 神经网络的参数
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST 数据集共有10个数字(0-9 )
dropout = 0.25 # 设置Dropout率


# 构建神经网络
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    # 命名空间
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']
        # 输入的TENSOR 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])
        # 定义卷基层1，激活函数为RELU
        # conv2d参数(inputs,filters,kernel_size,strides=(1, 1))
        #输入数据28*28，输出32通道，卷积核大小为5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # 池化层,滑动尺寸为2*2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # 卷基层2，输出64层，卷积核3
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # 池化层，互动尺寸2*2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        # 将池化层展开为1为的张量
        fc1 = tf.contrib.layers.flatten(conv2)
        # dense全连接神经网络
        fc1 = tf.layers.dense(fc1, 1024)
        # 设置的dropout，只在训练的时候设置
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # 输出
        out = tf.layers.dense(fc1, n_classes)

    return out

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# 
def model_fn(features, labels, mode):
    # 正向传播结果
    logits_train = conv_net(features, num_classes, dropout, reuse=False,
                            is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True,
                           is_training=False)

    # 预测的分类y_predict
    pred_classes = tf.argmax(logits_test, axis=1)
    #定义softmax
    pred_probas = tf.nn.softmax(logits_test)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # 损失函数
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    # 计算准确度
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [3]:

# Build the Estimator
model = tf.estimator.Estimator(model_fn)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# 开始训练
model.train(input_fn, steps=num_steps)

# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmppbw_os9d', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMINI~1\AppData\Local\Temp\tmppbw_os9d\model.ckpt.
INFO:tensorflow:loss = 2.32216, step = 1
INFO:tensorflow:global_step/sec: 4.93628
INFO:tensorflow:loss = 0.181068, step = 101 (20.241 sec)
INFO:tensorflow:Saving checkpoints for 200 into C:\Users\ADMINI~1\AppData\Local\Temp\tmppbw_os9d\model.ckpt.
INFO:tensorflow:Loss for final step: 0.130846.
INFO:tensorflow:Starting evaluation at 2018-05-04-14:55:54
INFO:tensorflow:Restoring parameters from C:\Users\ADMINI~1\AppData\Local\Temp\tmppbw_os9d\model.ckpt-200
INFO:tensorflow:Finished evaluation at 20